# 8.2 안전 운전자 예측 경진대회 탐색적 데이터 분석

- [안전 운전자 예측 경진대회 링크](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)
- [탐색적 데이터 분석 코드 참고 링크](https://www.kaggle.com/bertcarremans/data-preparation-exploration)

## 8.2.1 데이터 둘러보기

In [ ]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/2024-2-data-solution-ch-8/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
train['ps_reg_01']

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
train.info()

In [ ]:
import numpy as np
import missingno as msno

# 훈련 데이터 복사본에서 -1을 np.NaN로 변환
train_copy = train.copy().replace(-1, np.NaN)

# 결측값 시각화(처음 28개만)
msno.bar(df=train_copy.iloc[:, 1:29], figsize=(13, 6));

In [ ]:
msno.bar(df=train_copy.iloc[:, 29:], figsize=(13, 6));

In [ ]:
msno.matrix(df=train_copy.iloc[:, 1:29], figsize=(13, 6));

###  피처 요약표

In [ ]:
def resumetable(df):
    print(f'데이터 세트 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary['결측값 개수'] = (df == -1).sum().values # 피처별 -1 개수
    summary['고윳값 개수'] = df.nunique().values
    summary['데이터 종류'] = None
    for col in df.columns:
        if 'bin' in col or col == 'target':
            summary.loc[col, '데이터 종류'] = '이진형'
        elif 'cat' in col:
            summary.loc[col, '데이터 종류'] = '명목형'
        elif df[col].dtype == float:
            summary.loc[col, '데이터 종류'] = '연속형'
        elif df[col].dtype == int:
            summary.loc[col, '데이터 종류'] = '순서형'

    return summary

In [ ]:
summary = resumetable(train)
summary

In [ ]:
summary[summary['데이터 종류'] == '명목형'].index

In [ ]:
summary[summary['데이터 타입'] == 'float64'].index

## 8.2.2 데이터 시각화

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

### 타깃 값 분포

In [ ]:
def write_percent(ax, total_size):
    '''도형 객체를 순회하며 막대 그래프 상단에 타깃값 비율 표시'''
    for patch in ax.patches:
        height = patch.get_height()     # 도형 높이(데이터 개수)
        width = patch.get_width()       # 도형 너비
        left_coord = patch.get_x()      # 도형 왼쪽 테두리의 x축 위치
        percent = height/total_size*100 # 타깃값 비율
        
        # (x, y) 좌표에 텍스트 입력
        ax.text(left_coord + width/2.0,     # x축 위치
                height + total_size*0.001,  # y축 위치
                '{:1.1f}%'.format(percent), # 입력 텍스트
                ha='center')                # 가운데 정렬
    
mpl.rc('font', size=15)
plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train)) # 비율 표시
ax.set_title('Target Distribution');

### 이진 피처

In [ ]:
import matplotlib.gridspec as gridspec

def plot_target_ratio_by_features(df, features, num_rows, num_cols, 
                                  size=(12, 18)):
    mpl.rc('font', size=9) 
    plt.figure(figsize=size)                     # 전체 Figure 크기 설정
    grid = gridspec.GridSpec(num_rows, num_cols) # 서브플롯 배치
    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 좌우/상하 여백 설정

    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        # ax축에 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
        sns.barplot(x=feature, y='target', data=df, palette='Set2', ax=ax)

In [ ]:
bin_features = summary[summary['데이터 종류'] == '이진형'].index # 이진 피처
# 이진 피처 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
plot_target_ratio_by_features(train, bin_features, 6, 3) # 6행 3열 배치

### 명목형 피처

In [ ]:
nom_features = summary[summary['데이터 종류'] == '명목형'].index # 명목형 피처

plot_target_ratio_by_features(train, nom_features, 7, 2) # 7행 2열

### 순서형 피처

In [ ]:
ord_features = summary[summary['데이터 종류'] == '순서형'].index # 순서형 피처

plot_target_ratio_by_features(train, ord_features, 8, 2, (12, 20)) # 8행 2열

### 연속형 피처

In [ ]:
cont_features = summary[summary['데이터 종류'] == '연속형'].index # 연속형 피처

plt.figure(figsize=(12, 16))                # Figure 크기 설정
grid = gridspec.GridSpec(5, 2)              # GridSpec 객체 생성
plt.subplots_adjust(wspace=0.2, hspace=0.4) # 서브플롯 간 여백 설정

for idx, cont_feature in enumerate(cont_features):
    # 값을 5개 구간으로 나누기
    train[cont_feature] = pd.cut(train[cont_feature], 5)

    ax = plt.subplot(grid[idx])                # 분포도를 그릴 서브플롯 설정
    sns.barplot(x=cont_feature, y='target', data=train, palette='Set2', ax=ax)
    ax.tick_params(axis='x', labelrotation=10) # x축 라벨 회전

In [ ]:
train_copy = train_copy.dropna() # np.NaN 값 삭제

In [ ]:
plt.figure(figsize=(10, 8))
cont_corr = train_copy[cont_features].corr()     # 연속형 피처 간 상관관계 
sns.heatmap(cont_corr, annot=True, cmap='OrRd'); # 히트맵 그리기

# 8.3 안전 운전자 예측 경진대회 베이스라인 모델

## 8.3.1 피처 엔지니어링
### 데이터 합치기

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거

In [ ]:
all_features = all_data.columns # 전체 피처
all_features

### 명목형 피처 원-핫 인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature] 

onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

encoded_cat_matrix

### 필요 없는 피처 제거

In [ ]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# '1) 명목형 피처, 2) calc 분류의 피처, 3) 추가 제거할 피처'를 제외한 피처
remaining_features = [feature for feature in all_features 
                      if ('cat' not in feature and 
                          'calc' not in feature and 
                          'reg' not in feature and
                          'car' not in feature and
                          feature not in drop_features)]
remaining_features

In [ ]:
all_data[remaining_features]

## 데이터 인코딩 & 매트릭스화

In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

### 데이터 나누기

In [ ]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

## 8.3.2 평가지표 계산 함수 작성
### 정규화 지니계수 계산 함수

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]                      # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)       # 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)       # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True # 반환값

## 8.3.3 모델 훈련 및 성능 검증
### OOF 방식으로 LightGBM 훈련

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise': True,
          'random_state': 0}

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성 
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋

    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=params,        # 훈련용 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=100,  # 부스팅 반복 횟수 (기존 1000)
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval=gini,           # 검증용 평가지표
                          callbacks=[early_stopping(stopping_rounds=50)]) # 조기종료 조건
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수 
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

In [ ]:
print('OOF 검증 데이터 지니계수:', eval_gini(y, oof_val_preds))

## 8.3.4 예측 및 결과 제출

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('baseline_submission.csv')

In [ ]:
X

## 실습 및 과제 1.1 - 성능개선 1 (LightGBM 모델 with 피처엔지니어링, 하이퍼파라미터 최적화)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import lightgbm as lgb
from itertools import product

# 하이퍼파라미터 
param_grid = {
    'learning_rate': [0.01, 0.05],
    'num_leaves': [20, 30],
    'max_depth': [3, 5],
    'min_child_weight': [1, 5],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'lambda_l1': [1e-3],
    'lambda_l2': [1e-3],
}

# 하이퍼파라미터 조합 생성
param_combinations = list(product(
    param_grid['learning_rate'],
    param_grid['num_leaves'],
    param_grid['max_depth'],
    param_grid['min_child_weight'],
    param_grid['subsample'],
    param_grid['colsample_bytree'],
    param_grid['lambda_l1'],
    param_grid['lambda_l2'],
))

# Grid Search 설정
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=1991)
best_gini = -np.inf
best_params = None

# OOF 방식으로 각 하이퍼파라미터 조합 평가
for params in param_combinations:
    params_dict = {
        'objective': 'binary',
        'learning_rate': params[0],
        'num_leaves': params[1],
        'max_depth': params[2],
        'min_child_weight': params[3],
        'subsample': params[4],
        'colsample_bytree': params[5],
        'lambda_l1': params[6],
        'lambda_l2': params[7],
        'force_row_wise': True,
        'random_state': 0
    }

    oof_val_preds = np.zeros(X.shape[0])
    scores = []

    for train_idx, valid_idx in folds.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]

        dtrain = lgb.Dataset(X_train, y_train)
        dvalid = lgb.Dataset(X_valid, y_valid)

        # LightGBM 모델 학습
        model = lgb.train(
            params=params_dict,
            train_set=dtrain,
            num_boost_round=50,  # 부스팅 반복 횟수 축소
            valid_sets=dvalid,
            feval=gini,
            callbacks=[
                lgb.early_stopping(stopping_rounds=10),  # 조기 종료 조건
                lgb.log_evaluation(period=10)  # 학습 로그 출력 제어
            ]
        )

        # 검증 데이터 예측 및 평가
        preds = model.predict(X_valid)
        oof_val_preds[valid_idx] = preds
        scores.append(eval_gini(y_valid, preds))

    # 평균 Gini 계수 계산
    mean_gini = np.mean(scores)
    if mean_gini > best_gini:
        best_gini = mean_gini
        best_params = params_dict

# 최적의 하이퍼파라미터 출력
print(f"Best Gini Coefficient: {best_gini:.4f}")
print(f"Best Parameters: {best_params}")

# 최적의 하이퍼파라미터로 최종 모델 학습
oof_test_preds = np.zeros(X_test.shape[0])
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=1991)

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    # LightGBM 모델 학습
    model = lgb.train(
        params=best_params,
        train_set=dtrain,
        num_boost_round=50,
        valid_sets=dvalid,
        feval=gini,
        callbacks=[
            lgb.early_stopping(stopping_rounds=10),
            lgb.log_evaluation(period=10)
        ]
    )

    # 테스트 데이터 예측
    oof_test_preds += model.predict(X_test) / folds.n_splits
    oof_val_preds[valid_idx] += model.predict(X_valid)

# 최종 Gini 계수 출력
final_gini = eval_gini(y, oof_val_preds)
print(f"최종 OOF Gini Coefficient: {final_gini:.4f}")


## 실습 및 과제 1.2 - 성능개선 2 (XGBoost 모델)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import xgboost as xgb

# XGBoost 하이퍼파라미터 설정
xgb_params = {
    'objective': 'binary:logistic',  # 이진 분류
    'learning_rate': 0.05,           # 학습률
    'max_depth': 5,                  # 최대 트리 깊이
    'subsample': 0.8,                # 데이터 샘플링 비율
    'colsample_bytree': 0.8,         # 특성 샘플링 비율
    'random_state': 42,              # 랜덤 시드
}

# OOF 방식 예측값 저장
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

# Stratified K-Fold 설정
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# OOF 방식 학습
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print(f"Fold {idx+1}/{folds.n_splits}")

    # 학습 및 검증 데이터 분리
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # XGBoost 전용 DMatrix 생성
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    dtest = xgb.DMatrix(X_test)

    # XGBoost 모델 학습
    xgb_model = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=100,  # 부스팅 반복 횟수
        evals=[(dvalid, 'valid')],
        early_stopping_rounds=10,  # 조기 종료 조건
        verbose_eval=10
    )

    # OOF 예측
    oof_val_preds[valid_idx] = xgb_model.predict(dvalid)
    oof_test_preds += xgb_model.predict(dtest) / folds.n_splits

    # Fold 별 Gini 계수 계산
    fold_gini = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f"Fold {idx+1} Gini Coefficient: {fold_gini:.4f}")

# 최종 OOF Gini 계수
final_gini = eval_gini(y, oof_val_preds)
print(f"Final OOF Gini Coefficient: {final_gini:.4f}")


## 실습 및 과제 1.3 - LightGBM과 XGBoost 앙상블

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import lightgbm as lgb
import xgboost as xgb

# OOF 예측값 초기화
oof_val_preds_lgb = np.zeros(X.shape[0])
oof_test_preds_lgb = np.zeros(X_test.shape[0])
oof_val_preds_xgb = np.zeros(X.shape[0])
oof_test_preds_xgb = np.zeros(X_test.shape[0])

# 하이퍼파라미터 설정
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

lgb_params = {
    'objective': 'binary',
    'learning_rate': 0.05,
    'num_leaves': 30,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

# Stratified K-Fold 설정
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# OOF 방식으로 두 모델 학습
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print(f"Fold {idx+1}/{folds.n_splits}")

    # 데이터 분리
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # XGBoost 학습
    dtrain_xgb = xgb.DMatrix(X_train, label=y_train)
    dvalid_xgb = xgb.DMatrix(X_valid, label=y_valid)
    dtest_xgb = xgb.DMatrix(X_test)

    xgb_model = xgb.train(
        params=xgb_params,
        dtrain=dtrain_xgb,
        num_boost_round=100,
        evals=[(dvalid_xgb, 'valid')],
        early_stopping_rounds=10,
        verbose_eval=10
    )
    
    # OOF 예측
    oof_val_preds_xgb[valid_idx] = xgb_model.predict(dvalid_xgb)
    oof_test_preds_xgb += xgb_model.predict(dtest_xgb) / folds.n_splits

    # LightGBM 학습
    dtrain_lgb = lgb.Dataset(X_train, y_train)
    dvalid_lgb = lgb.Dataset(X_valid, y_valid, reference=dtrain_lgb)

    # Callbacks 사용하여 early_stopping 적용
    callbacks = [lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(period=10)]

    lgb_model = lgb.train(
        params=lgb_params,
        train_set=dtrain_lgb,
        num_boost_round=100,
        valid_sets=[dvalid_lgb],
        callbacks=callbacks
    )

    # OOF 예측
    oof_val_preds_lgb[valid_idx] = lgb_model.predict(X_valid)
    oof_test_preds_lgb += lgb_model.predict(X_test) / folds.n_splits

    # 각 모델의 Gini 계수 출력 (기존 eval_gini 사용)
    gini_xgb = eval_gini(y_valid, oof_val_preds_xgb[valid_idx])
    gini_lgb = eval_gini(y_valid, oof_val_preds_lgb[valid_idx])
    print(f"Fold {idx+1} - XGBoost Gini: {gini_xgb:.4f}, LightGBM Gini: {gini_lgb:.4f}")

# 앙상블 예측 (두 모델의 예측 평균)
oof_val_preds_ensemble = (oof_val_preds_lgb + oof_val_preds_xgb) / 2
oof_test_preds_ensemble = (oof_test_preds_lgb + oof_test_preds_xgb) / 2

# 최종 Gini 계수 출력 (앙상블 모델)
final_gini_ensemble = eval_gini(y, oof_val_preds_ensemble)
print(f"Final OOF Gini Coefficient (Ensemble): {final_gini_ensemble:.4f}")


## 과제2 - 실습 및 과제 1.1~3에서 제외한 나만의 성능개선 방법을 시도해보고 이에 대해 설명 하라


XGBOOST 최적 파라미터 찾기

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.metrics import make_scorer
# Gini 계수를 평가 지표로 사용하기 위한 함수 정의 
gini_scorer = make_scorer(eval_gini, greater_is_better=True)

# XGBoost 하이퍼파라미터 범위 설정 
xgb_param_dist = {
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(3, 8), 
    'subsample': uniform(0.6, 0.4),  
    'colsample_bytree': uniform(0.6, 0.4), 
    'n_estimators': [50, 100, 150]
}

# XGBoost 모델 정의
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=42)

# RandomizedSearchCV로 XGBoost 최적화
xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_dist,
    n_iter=10,  # 10개의 조합을 랜덤하게 탐색
    cv=3,
    scoring=gini_scorer,  # Gini 계수를 평가 기준으로 사용
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# XGBoost 모델 최적화
xgb_random_search.fit(X, y)

# 최적 하이퍼파라미터 및 Gini 계수 출력
print("XGBoost Best Parameters:", xgb_random_search.best_params_)
print("XGBoost Best Gini Coefficient: {:.4f}".format(xgb_random_search.best_score_))


In [ ]:
#your code here
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.base import BaseEstimator, TransformerMixin



# LightGBM 하이퍼파라미터 (이전에 최적화된 값)
lgb_params = {
    'objective': 'binary',
    'learning_rate': 0.05,
    'num_leaves': 30,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

# XGBoost 하이퍼파라미터
xgb_params = {
    'objective': 'binary:logistic',
    'learning_rate': 0.10656320330745593,
    'max_depth': 4,
    'subsample': 0.7218455076693483,
    'colsample_bytree': 0.9795542149013333,
    'n_estimators': 100,
    'random_state': 42
}

# Stratified K-Fold 설정
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 메타 모델인 Logistic Regression 준비
meta_model = LogisticRegression(random_state=42)

# 스태킹을 위한 기본 모델들
base_learners = [
    ('xgb', xgb.XGBClassifier(**xgb_params)),
    ('lgb', lgb.LGBMClassifier(**lgb_params))
]

# StackingClassifier로 스태킹 모델 구성
stacking_model = StackingClassifier(
    estimators=base_learners, 
    final_estimator=meta_model, 
    cv=3
)

# OOF 예측값 초기화
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

# OOF 방식으로 모델 학습 및 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print(f"Fold {idx+1}/{folds.n_splits}")

    # 데이터 분리
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # Stacking 모델 학습
    stacking_model.fit(X_train, y_train)

    # OOF 예측
    oof_val_preds[valid_idx] = stacking_model.predict_proba(X_valid)[:, 1]
    oof_test_preds += stacking_model.predict_proba(X_test)[:, 1] / folds.n_splits

    # Gini 계수 계산
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f"Fold {idx+1} Gini: {gini_score:.4f}")

# 최종 Gini 계수 출력 (앙상블 모델)
final_gini = eval_gini(y, oof_val_preds)
print(f"Final OOF Gini Coefficient (Stacking): {final_gini:.4f}")


In [ ]:
print(f"훈련 데이터 특성 수: {X.shape}")
print(f"테스트 데이터 특성 수: {X_test.shape}")

In [ ]:
import pandas as pd


# 예측값 생성
oof_test_preds = stacking_model.predict_proba(X_test)[:, 1]  # stacking_model은 최종 앙상블 모델

# 예측 결과를 sample_submission 형식에 맞게 저장
submission['target'] = oof_test_preds  # 예측값 추가

# 제출 파일로 저장
submission.to_csv('submission.csv')  # index=False로 인덱스 제외

print("Submission file created successfully: 'submission.csv'")


In [ ]:
submission